In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
import mygene
from  TCGA_files import queryFile
from matplotlib import pyplot as plt

In [ ]:
mg = mygene.MyGeneInfo()

In [ ]:
#mg.getgene("ENSG00000135094", 'name,symbol,refseq.rna,type_of_gene,exac.bp,genomic_pos')

In [ ]:
working_dir = "/Users/filippo/Developer/tesi"
os.chdir(working_dir)
dirs = os.listdir("data")

In [ ]:
len(dirs)

In [ ]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=[0], index_col=[0])
df.columns=["type_of_gene"]
print(len(df))
df.head()

In [ ]:
uniq = [g[:15] for g in df.index.values]

In [ ]:
len(np.unique(uniq))

In [ ]:
#q_many = mg.getgenes(df[df['type_of_gene']=='protein-coding'].index.values,'symbol,genomic_pos,genomic_pos_hg19')

In [ ]:
q_many = pd.read_csv("genes_l.txt", index_col=[0], header=[0])
#df_lenghts = df[df['type_of_gene']=='protein-coding']
lenghts = q_many['lenght']
df_lenghts = df
df_lenghts.insert(0,'lenght',lenghts)
df_lenghts.to_csv("genes_l.txt",header=True,index=True)

In [ ]:
df = pd.read_csv(("%s/mainTable.csv"%working_dir), index_col=[0])

In [ ]:
df.head()

In [ ]:
ngenes = len(df.index)
nfiles = len(df.columns)
print("genes:%d\trealizations:%d"%(ngenes,nfiles))

## Counts to RPK
RPK=$\frac{counts}{lenght}$

In [ ]:
df = pd.concat((df,q_many), ignore_index=False, sort=False, axis=1)
df.drop('type_of_gene',axis=1,inplace=True)
df.head()

In [ ]:
df = df.div(df['lenght'],axis=0)
df = df.mul(1000,axis=0) #kilobase
df.drop('lenght', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv("mainTable_RPK.csv",index=True, header=True)

### null

In [ ]:
df_null=pd.read_csv("nullTable.csv",header=None, index_col=None)
df_null.insert(0,'lenght',q_many['lenght'].values)
df_null.head()

In [ ]:
df_null = df_null.div(df_null['lenght'],axis=0).mul(1000,axis=0) #kilobase
df_null.drop('lenght',axis=1, inplace=True)

In [ ]:
df_null.head()

In [ ]:
df_null.to_csv("nullTable.csv", header=False, index=False)

## Means sigmas
thresholds are applied for FPKM between 0.1 and 1e5

In [ ]:
%timeit
means = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    means.append(np.average(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
%timeit
variances = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    variances.append(np.var(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
df_mv = pd.DataFrame(data=np.array(means), columns=['mean'], index=df.loc[:,df.keys()[0]].values)
df_mv.insert(1, 'variance', np.array(variances))
df_mv.head()
df_mv.to_csv("meanVariances.csv")

## manipulate meansigmas.csv

In [ ]:
df_mv = pd.read_csv("meanVariances.csv", index_col = [0])
df_mv.head()

### Add type of gene

In [ ]:
gdf = pd.read_csv("genes.txt", index_col=[0])
gdf.head()

In [ ]:
len(gdf)

In [ ]:
df_mv = df_mv.drop(labels='type_of_gene', axis=1)
df_mv.insert(1,'type_of_gene', '')
#for g in df_mv[[df_mv['type_of_gene']==type_of_gene]].index.values
for g in df_mv.index.values:
    df_mv.at[g, 'type_of_gene'] = gdf.loc[g[:15]]['type_of_gene']

In [ ]:
#df_mv = df_mv[df_mv['type_of_gene']==type_of_gene]
df_mv.to_csv("meanVariances.csv", index=True, header=True)

### Add occurence

In [ ]:
odf = pd.read_csv("O.dat", header=None)
#print(len(df_mv[df_mv['type_of_gene']=='protein-coding']))
odf.head()

In [ ]:
len(np.array(odf.loc[:,0].values*5000))

In [ ]:
type_of_gene = 'protein-coding'

In [ ]:
len(gdf[gdf['type_of_gene']==type_of_gene].index)

In [ ]:
len(df_mv.index)

In [ ]:
gdfpc = gdf.loc[gdf['type_of_gene']==type_of_gene]
gdfpc.insert(1, 'occurrence', np.array(odf.loc[:,0].values*5000))
#df_mv.insert(3,'occurrence', 0)
for g in df_mv.index.values:
    df_mv.at[g, 'occurrence'] = gdfpc.loc[g[:15]]['occurrence']

In [ ]:
df_mv.head()

In [ ]:
df_mv.to_csv("meanVariances.csv", header=True, index=True)

## clean floating points

In [ ]:
df.to_csv("mainTable.csv", index=False, header=True, float_format="%.1f")

# File list and properties

In [ ]:
#df_files = pd.read_csv("files.txt")
#df_files = pd.DataFrame(index=[sample[:36] for sample in df_files.columns.values[1:]])

In [ ]:
#df_files = pd.read_csv("files.txt", index_col=[0], header=None)
df_files.insert(0,'primary_site','')
df_files.insert(1,'disease_type', '')
df_files.head()

In [ ]:
#for sample in df_files.index:
for sample in df_files[df_files['primary_site']==''].index:
    try:
        sdf=queryFile(sample+'.FPKM.txt.gz')
        df_files.at[sample,'primary_site'] = sdf['primary_site'][0]
        df_files.at[sample, 'disease_type'] = sdf['disease_type'][0]
    except:
        for i in range(10):
            try:
                sdf=queryFile(sample+'%1.0f.FPKM.txt'%i)
                df_files.drop(labels=sample, axis=0, inplace=True)
                sample = sample+'%d'%i
                df_files.at[sample,'primary_site'] = sdf['primary_site'][0]
                df_files.at[sample, 'disease_type'] = sdf['disease_type'][0]
                break
            except:
                continue

In [ ]:
df_files.to_csv("files.txt", index=True, header=True)